In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.image as img
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,rmsprop,Adadelta
from keras.metrics import categorical_crossentropy
from batch_generator import batch_generator
from metrics import (dice_coef,dice_loss,categorical_loss)
from models import create
from image_generator import generate_image
from glob import glob
import os


In [3]:
samples_paths=sorted(glob("./Data/Train/samples_processed/*.npy"))
masks_paths=sorted(glob("./Data/Train/masks_reformatted/*.npy"))


In [4]:
samples_paths

['./Data/Train/samples_processed\\sample_001.npy',
 './Data/Train/samples_processed\\sample_002.npy',
 './Data/Train/samples_processed\\sample_003.npy',
 './Data/Train/samples_processed\\sample_004.npy',
 './Data/Train/samples_processed\\sample_005.npy',
 './Data/Train/samples_processed\\sample_006.npy',
 './Data/Train/samples_processed\\sample_007.npy',
 './Data/Train/samples_processed\\sample_008.npy',
 './Data/Train/samples_processed\\sample_009.npy',
 './Data/Train/samples_processed\\sample_010.npy',
 './Data/Train/samples_processed\\sample_011.npy',
 './Data/Train/samples_processed\\sample_012.npy',
 './Data/Train/samples_processed\\sample_013.npy',
 './Data/Train/samples_processed\\sample_014.npy',
 './Data/Train/samples_processed\\sample_015.npy',
 './Data/Train/samples_processed\\sample_016.npy',
 './Data/Train/samples_processed\\sample_017.npy',
 './Data/Train/samples_processed\\sample_018.npy',
 './Data/Train/samples_processed\\sample_019.npy',
 './Data/Train/samples_processe

In [5]:
train_sample_paths,test_sample_paths=train_test_split(samples_paths,test_size=0.25)

In [6]:
traingen=batch_generator(train_sample_paths,32) #Mini batch size of 16
testgen=batch_generator(test_sample_paths,32)


In [7]:
model=create()

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 128, 3  0         
                             )]                                  
                                                                 
 conv3d (Conv3D)             (None, 63, 63, 63, 8)     656       
                                                                 
 conv3d_1 (Conv3D)           (None, 31, 31, 31, 16)    3472      
                                                                 
 conv3d_2 (Conv3D)           (None, 15, 15, 15, 32)    13856     
                                                                 
 conv3d_transpose (Conv3DTra  (None, 31, 31, 31, 32)   27680     
 nspose)                                                         
                                                                 
 conv3d_transpose_1 (Conv3DT  (None, 63, 63, 63, 16)   13840 

In [9]:
sample,mask=next(traingen)

In [10]:
sample.shape

(32, 128, 128, 128, 3)

In [11]:
mask.shape

(32, 128, 128, 128, 4)

In [12]:
sample

array([[[[[ 4.100e-01,  3.070e-01,  3.080e-01],
          [ 3.670e-01,  2.750e-01,  2.850e-01],
          [ 4.400e-02,  2.300e-01,  2.390e-01],
          ...,
          [ 1.270e-01,  3.700e-01,  4.450e-01],
          [ 3.450e-01,  3.990e-01,  4.690e-01],
          [ 3.900e-01,  4.160e-01,  4.770e-01]],

         [[ 3.400e-01,  2.680e-01,  2.770e-01],
          [ 3.500e-01,  2.420e-01,  2.600e-01],
          [ 3.660e-01,  2.020e-01,  2.250e-01],
          ...,
          [ 2.480e-01,  2.740e-01,  3.240e-01],
          [ 3.440e-01,  3.640e-01,  4.540e-01],
          [ 3.750e-01,  3.890e-01,  4.690e-01]],

         [[-1.510e-01,  2.330e-01,  2.320e-01],
          [ 3.080e-01,  2.040e-01,  2.210e-01],
          [ 4.600e-01,  1.290e-01,  1.930e-01],
          ...,
          [ 3.640e-01, -1.010e-01, -1.980e-01],
          [ 1.610e-01, -2.000e-02, -2.400e-02],
          [-1.360e-01,  1.340e-01,  2.840e-01]],

         ...,

         [[ 1.090e-01,  1.000e-01,  5.800e-02],
          [ 1.030e-01,

In [13]:
#Compute steps per epoch
def steps(m,batchsize):
    return (m+batchsize-1)//batchsize

In [14]:
train_steps=steps(len(train_sample_paths),32)
test_steps=steps(len(test_sample_paths),32)


In [15]:
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss=tf.keras.losses.CategoricalCrossentropy(),metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [16]:
history=model.fit(traingen,steps_per_epoch=train_steps,epochs=1,validation_data=testgen,validation_steps=test_steps,verbose=1,max_queue_size=1)


: 

: 

In [66]:
l=[]
for i in range(369):
    sno="{0:0=3d}".format(i+1)
    s=np.load("./Data/Train/samples_processed/sample_"+sno+".npy")

    l.append(s.shape)
